In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt

# Exercice 9: Couche limite

Le système suivant illustre la notion de couche limite :
    
${\large \begin{equation*}
\epsilon \frac{d^{2} y}{dx^{2}}-x^{2}\frac{dy}{dx}-y=0, \, 0<x<1, \, \epsilon =10^{-2},
\end{equation*}}$

avec les conditions frontières: $y(0)=y(1)=1$.

Condition d'unicité: $N>49$

Système linéaire à résoudre:

$
A=\begin{pmatrix}
a_{1} & c_{1} & 0 &\cdots & \cdots &  0\\
b_{2} & a_{2} & c_{2} & 0 & \cdots & 0  \\
0 & \ddots & \ddots & \ddots &  \ddots &\vdots  \\
\vdots & \ddots  & \ddots & \ddots & \ddots & 0  \\
0 & \cdots  & 0& b_{N-1} & a_{N-1} & c_{N-1}  \\
0 & \cdots  & 0 & 0 & b_{N} & a_{N} 
\end{pmatrix}
\begin{pmatrix}
y_{1}\\
y_{2}\\
\vdots\\
y_{N-1}\\
y_{N}\\
\end{pmatrix}
=\begin{pmatrix}
-b_{1}\\
0\\
\vdots\\
0\\
-c_{N}\\
\end{pmatrix}
$

avec

$
\left\{
                \begin{array}{ll}
                  a_{i}=-2-\frac{h^2}{\epsilon}\\
                  b_{i}=1+\frac{hx_{i}^{2}}{2\epsilon}\\
                  c_{i}=1-\frac{hx_{i}^{2}}{2\epsilon}
                \end{array}
              \right.
$



Conditionnement de la matrice:

In [2]:
N=1
eps=10**(-2)
x=np.linspace(0.,1.,N+2)
y=np.zeros(N+2)
h=x[1]-x[0]
ybound=[1,1]
xbis=x[1:-1]
a=-2.-(h**2)/eps*np.ones(N)
b=1.+h*(xbis**2)/2./eps
c=1.-h*(xbis**2)/2./eps
A=np.diagflat(b[1:],-1)+np.diagflat(c[:-1],1)+np.diagflat(a)
#print(A)
w,v=np.linalg.eig(A.T@A)
cond=np.sqrt(np.max(np.abs(w))/np.min(np.abs(w)))
print('Conditionnement de la matrice %.3f'%(cond))

Conditionnement de la matrice 1.000


In [3]:
def tridiag(a,b,c,f):
    N=f.size
    x=np.zeros(N)
    cstar=np.zeros(N)
    astar=a[0]
    x[0]=f[0]/astar
    for k in range(1,N):
        cstar[k-1]=c[k-1]/astar
        astar=a[k]-b[k]*cstar[k-1]
        x[k]=(f[k]-b[k]*x[k-1])/astar
    for k in range(N-2,-1,-1):
        x[k]-=cstar[k]*x[k+1]
    return(x)


def c_lim(N):
    eps=10**(-2)
    x=np.linspace(0.,1.,N+2)
    y=np.zeros(N+2)
    h=x[1]-x[0]
    ybound=[1,1]
    xbis=x[1:-1]
    a=-2.-(h**2)/eps*np.ones(N)
    b=1.+h*(xbis**2)/2./eps
    c=1.-h*(xbis**2)/2./eps
    z=np.zeros(N)
    z[0]=-b[0]*ybound[0]
    z[-1]=-c[-1]*ybound[1]
    y[1:-1]=tridiag(a,b,c,z)
    y[0]=ybound[0]
    y[-1]=ybound[1]
    return(x,y,cond)


In [4]:
x10,y10,cond10=c_lim(10)
x20,y20,cond20=c_lim(20)
x50,y50,cond50=c_lim(50)
x120,y120,cond120=c_lim(120)



In [5]:
fig,ax=plt.subplots(figsize=(5,5))
plt.plot(x10,y10,label='N=10',linestyle='--',marker='o',ms=7)
plt.plot(x20,y20,label='N=20',linestyle='--',marker='v',ms=7)
plt.plot(x50,y50,label='N=50',linestyle='--',marker='D',ms=7)
plt.plot(x120,y120,label='N=120')
plt.legend(loc='best',fontsize='small')
plt.xlabel('x')
plt.ylabel('y')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Exercice 11: équation de Bratu

Résoudre l'équation de Bratu ${\large y'' = −e^y}$ , ${\large 0 < x < 1}$, avec
${\large y (0) = y (1) = 0}$. Utiliser comme premier essai ${\large y_{0}(x) = x − x^{2}}$ ;
recommencez et utilisez ${\large y_{0}(x) = 16(x − x^{2})}$. 


>**Newton-Raphson**
>
>Système d'équations non linéaires
>
>
>${\large \mathbf{F}(\mathbf{y})=\mathbf{0}}$
>
>On a besoin de deux choses pour appliquer la méthode: ${\large \mathbf{y_{0}}}$ première estimation de la solution et ${\large J}$ le jacobien de ${\large \mathbf{F}(\mathbf{y})}$
>
>Le schéma suivant doit converger vers la solution si le système est suffisamment régulier:
>
>${\large \mathbf{y}_{k+1} = \mathbf{y}_{k} - J_{k}^{-1} \mathbf{F}_{k} \, \Rightarrow \, J_{k}\mathbf{y}_{k+1}  =J_{k}\mathbf{y}_{k}-\mathbf{F}_{k} \quad ,k=0,1,2,...}$





In [6]:
def Jac(y,h):
    N=y.size
    J=np.zeros((N,N))
    J[0,0]=-2.+(h**2)*np.exp(y[0])
    J[-1,-1]=-2.+(h**2)*np.exp(y[-1])
    J[0,1]=1.
    J[-1,-2]=1.
    for k in range(1,N-1):
        J[k,k]=-2.+(h**2)*np.exp(y[k])
        J[k,k-1]=1.
        J[k,k+1]=1.
    return(J)

def Fk(z,h):
    N=z.size
    F=np.zeros(N)
    F[0]=z[1]-2.*z[0]+(h**2)*np.exp(z[0])          # CF z_firstpoint=0
    F[-1]=-2.*z[-1]+z[-2]+(h**2)*np.exp(z[-1])# CF z_lastpoint=0
    for k in range(1,N-1):
        F[k]=z[k+1]-2.*z[k]+z[k-1]+(h**2)*np.exp(z[k])
    return(F)



def bratu(N,N_it,x,Y0):
    h=x[1]-x[0]
    Y=np.zeros((N,N_it))
    Y[:,0]=Y0
    for k in range(1,N_it):
        diag_J=-2.+(h**2)*np.exp(Y[:,k-1])
        rhs=np.dot(Jac(Y[:,k-1],h),Y[:,k-1])-Fk(Y[:,k-1],h)
        Y[:,k]=tridiag(diag_J,np.ones(N),np.ones(N),rhs)
    return(np.append(np.insert(Y[:,-1],0,0.),0.))




In [9]:
#paramètres d'intégration
N=10
N_int=5
x=np.linspace(0.,1,N+2)

#Solutions analytiques
c1=0.379
Y_anal1=-2.*np.log(np.cosh(c1*(1.-2.*np.linspace(0,1,101)))/np.cosh(c1))
c2=2.73;
Y_anal2=-2.*np.log(np.cosh(c2*(1.-2.*np.linspace(0,1,101)))/np.cosh(c2))

#Intégration numérique
Y1=bratu(N,N_int,x,Y0=(x[1:-1]-x[1:-1]**2))
Y2=bratu(N,N_int,x,Y0=16.*(x[1:-1]-x[1:-1]**2))

In [11]:
fig,ax=plt.subplots(figsize=(7,7))
ax.plot(x,Y1,'^b',ms=7,label='$y_0=(x-x^2)$')
ax.plot(x,Y2,'^r',ms=7,label='$y_0=16(x-x^2)$')
ax.plot(np.linspace(0,1,101),Y_anal1,'b',label='$c=0.379$')
ax.plot(np.linspace(0,1,101),Y_anal2,'r',label='$c=2.73$')
ax.legend(loc='best',fontsize='small')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …